In [ ]:
!pip install opendatasets

In [ ]:
!pip install tensorflow tensorflow-addons datasets transformers torch

In [ ]:
!pip install visualkeras

In [ ]:
import visualkeras
import tensorflow as tf
import matplotlib.pyplot as plt
import opendatasets as od
import tensorflow as tf
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import EfficientNetB2, InceptionV3
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
od.download(
    "/kaggle/input/breast-mammography-private")

In [ ]:
BATCH_SIZE = 64
IMG_SIZE = (224, 224)
train_dir = "/kaggle/input/breast-mammography-private/train"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)


# Define the label mapping
label_map = {0: "NORMAL", 1: "ABNORMAL"}

# Collect 5 images for each class
images_by_class = {0: [], 1: []}

for image, label in train_dataset.unbatch().take(1000):  # Increase the range if necessary
    label_value = label.numpy()
    if len(images_by_class[label_value]) < 5:
        images_by_class[label_value].append((image, label))
    if all(len(images) >= 5 for images in images_by_class.values()):
        break

# Combine the images and labels
images_to_plot = []
for class_images in images_by_class.values():
    images_to_plot.extend(class_images)

# Display the images
plt.figure(figsize=(20, 10))
for i, (img, lbl) in enumerate(images_to_plot):
    plt.subplot(4, 5, i + 1)  # 4 rows, 5 columns
    plt.imshow(img.numpy().astype("uint8"))
    label = lbl.numpy()
    plt.title(f"{label_map[label]} ({label}), Size: {img.shape}")
    plt.axis("off")
plt.show()

In [ ]:
# Parameters
BATCH_SIZE = 64
IMG_SIZE = (224, 224)
PATCH_SIZE = 16
NUM_CLASSES = 2
EPOCHS = 50
LEARNING_RATE = 3e-4  # Adjusted learning rate

# Load the dataset
train_dir = "/kaggle/input/breast-mammography-private/train"
test_dir = "/kaggle/input/breast-mammography-private/test"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Preprocess the dataset
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(40 / 360),  # Convert degrees to fraction
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomTranslation(0.2, 0.2),  # width_shift_range and height_shift_range
])

In [ ]:
# Load pre-trained models without the top layers
MobileNet_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
DenseNet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Set trainable to False to freeze the pre-trained layers
MobileNet_model.trainable = False
DenseNet_model.trainable = False
VGG_model.trainable = False

In [ ]:
def create_ensemble_model(input_shape, num_classes):
    # Inputs
    inputs = layers.Input(shape=input_shape)

    # Data augmentation
    augmented = data_augmentation(inputs)

    # MobileNet model
    mobilenet_output = MobileNet_model(augmented)
    print(f"mobilenet_output: {mobilenet_output.shape}")
    mobilenet_output = layers.GlobalAveragePooling2D()(mobilenet_output)
    print(mobilenet_output.shape)

    # DenseNet model
    densenet_output = DenseNet_model(augmented)
    print(f"densenet_output: {densenet_output.shape}")
    densenet_output = layers.GlobalAveragePooling2D()(densenet_output)
    print(densenet_output.shape)

    # VGG model
    vgg_output = VGG_model(augmented)
    print(f"vgg_output: {vgg_output.shape}")
    vgg_output = layers.GlobalAveragePooling2D()(vgg_output)
    print(vgg_output.shape)

    # Concatenate outputs
    concatenated = layers.Concatenate()([mobilenet_output, densenet_output, vgg_output])
    print(f"concatenated: {concatenated.shape}")

    # Dense layers after concatenation
    x = layers.BatchNormalization()(concatenated)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.4)(x) # Test
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.3)(x) # Test

    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    print(outputs.shape)

    # Create model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create the ensemble model
input_shape = IMG_SIZE + (3,)
print(input_shape)
print(NUM_CLASSES)
model = create_ensemble_model(input_shape, NUM_CLASSES)

In [ ]:
# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Callbacks for early stopping, reducing learning rate, and saving the best model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="/kaggle/working/model/best_model.keras",
    save_best_only=True,
    monitor="val_sparse_categorical_accuracy",
    mode="max",
    verbose=1
)

# Train the model
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Assuming 'model' and 'test_dataset' are already defined

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# Get predictions and true labels
y_pred = []
y_true = []
y_prob = []

for images, labels in test_dataset:
    predictions = model.predict(images)
    y_pred.extend(tf.argmax(predictions, axis=1).numpy())
    y_true.extend(labels.numpy())
    y_prob.extend(predictions)

# Define target names for the classification report
target_names = ['NORMAL', 'ABNORMAL']

# Calculate and print the classification report
print(classification_report(y_true, y_pred, target_names=target_names))

# Calculate precision, recall, F1 score using sklearn
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Calculate AUC-ROC score
y_true_one_hot = tf.keras.utils.to_categorical(y_true, num_classes=2)
auc_roc = roc_auc_score(y_true_one_hot, y_prob, average='weighted', multi_class='ovr')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f'AUC-ROC: {auc_roc:.2f}')

In [ ]:
num_test_images = len(test_dataset) * BATCH_SIZE

# Count the number of samples for each class
normal_count = np.sum(np.array(y_true) == 0)
abnormal_count = np.sum(np.array(y_true) == 1)

# Print the counts for each class
print(f'Number of Normal samples: {normal_count}')
print(f'Number of Abnormal samples: {abnormal_count}')

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history.history['sparse_binary_accuracy'], label='Training Accuracy')
plt.plot(history.history['val_sparse_binary_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
model_load_path = 'VGG16_model.h5'
loaded_model = tf.keras.models.load_model(model_load_path)

print(f"Model loaded from: {model_load_path}")

In [ ]:
model_save_path = 'MobileNetV2_model.h5'
model.save(model_save_path)

print(f"Model saved at: {model_save_path}")

In [ ]:
model_load_path = 'DenseNet121_model.h5'
loaded_model = tf.keras.models.load_model(model_load_path)

print(f"Model loaded from: {model_load_path}")

In [ ]:
visualkeras.layered_view(loaded_model, legend=True)